# Automatisation des tâches d'administration système

# M3206 - TP 2 : Analyse de trace de connexion sur un serveur

Ceci est un canevas pour votre compte-rendu de TP, que vous devrez rendre sur [e-campus](http://e-campus.iut-velizy.uvsq.fr/). Vous modifierez directement dans ce document.

## Binôme

* Nom1 Prénom1
* Nom2 Prénom2

## Préambule

Le fichier de logs est un exemple tiré du fichier `/var/log/messages` généré par `syslogd`, le démon de gestion de log de GNU/Linux.

Les fichiers pour ce TP sont disponible sur e-campus, il s'agit de `test_messages100` et `test_messages5000`. Vérifiez le répertoire dans lequel est exécuté ce notebook et copiez-y ces deux fichiers. Comme le fichier `/var/log/messages` peut être de grande taille (> 10 Mo), nous travaillerons sur des fichiers plus petits, de 100 et 5000 lignes.

Pour ce TP, vous aurez besoin d'installer les bibliothèques suivantes : 
* pyparsing
* pandas
* basemap (facultatif)

Pour installer ces bibliothèques, si vous utilisez Anaconda, exécutez la ligne suivante:

    conda install pyparsing pandas basemap
    
Si vous utilisez le Python de CentOS 7, exécutez les lignes:

    yum install python-pip ipython
    pip install notebook

Comme vous aller créer des figures, nous allons demander à les afficher dans le corps du notebook :

In [1]:
% matplotlib inline

## Parseur de logs

Pour lire et interpréter (*to parse* en anglais) correctement le fichier de logs, on va utiliser un *parser*. Comme il n'y pas de *parser* déjà disponible pour le fichier `/var/log/messages`, il faut l'écrire. Nous allons utiliser le module `pyparsing` qui permet de définir son propre parser.

Nous allons créer une nouvelle classe de parseur que nous appellerons `SyslogParser`. Nous aurons alors besoin de définir la fonction d'initialisation, appellée quand on crée le parseur et la fonction pour parser un fichier. 

In [2]:
from pyparsing import Word, nums, alphas, string, Combine, Optional, Suppress, Regex, ParseException
 
class SyslogParser(object):
    """Parser for syslog information"""
    def __init__(self):
        """Parser for syslog information
        
        Initialize the parser for a file like /var/log/message, with lines of the form:
        timestamp hostname command[pid]: message
        """
        ints = Word(nums)
        # extract timestamp
        month = Word(string.ascii_uppercase, string.ascii_lowercase, exact=3)
        day = ints
        hour = Combine(ints + ":" + ints + ":" + ints)
        timestamp = month + day + hour
        # hostname
        hostname = Word(alphas + nums + "_" + "-" + ".")
        # appname
        command = Word(alphas + "/" + "-" + "_" + ".") + Optional(Suppress("[") + ints + Suppress("]")) + Suppress(":")
        # message
        message = Regex(".*")
        # pattern build
        self.__pattern = timestamp + hostname + command + message

    def parse(self, line):
        """Parser for syslog information
        
        Parse file like /var/log/message, with lines of the form:
        timestamp hostname command[pid]: message
        """
        payload = {}
        try:
            parsed = self.__pattern.parseString(line)
            payload["timestamp"] = parsed[0]+" "+parsed[1]+" "+parsed[2]
            payload["hostname"] = parsed[3]
            payload["command"] = parsed[4]
            payload["pid"] = parsed[5]
            payload["message"] = parsed[6]
        except ParseException:
            print ("error parsing line:", line)
            pass
        except:
            raise
        return payload
    
    def parseSSH(self, line):
        """Parser syslog for sshd information
        
        Parse files like /var/log/message, with lines of the form:
        timestamp hostname command[pid]: message
        Only the information relevant to sshd are kept.
        """
        payload = {}
        try:
            parsed = self.__pattern.parseString(line)
            if parsed[4] != 'sshd':
                return
            payload["timestamp"] = parsed[0]+" "+parsed[1]+" "+parsed[2]
            payload["pid"] = parsed[5]
            message = parsed[6].split(';')
            msg = []
            for f in message:
                msg.append(f.split(':'))
            if len(msg) == 1 or len(msg) == 2:
                if message[0].split()[0] == 'reverse':
                    payload['reversedMapping'] = False
                    return payload
                else:
                    return 
            if msg[1][1] == ' Version':
                payload["client"] = msg[4][1]
            if msg[1][1] == ' Authname':
                payload["user"] = msg[3][1].split('[')[0]
            payload['ip'] = msg[2][1]
        except ParseException:
            print ("error parsing line:", line)
            pass
        except:
            print ("error parsing line:", line)
            raise
        return payload

Après avoir importé les modules et fonctions dont nous avions besoin, nous avons crée la classe `SyslogParser` et nous lui avons associé deux fonctions `__init__(self)` (qui est obligatoire) et `parse(self, line)` qui nous permet de parser la ligne `line` passée en argument. 

La fonction `__init__(self)` utilise les outils fournis par le module `pyparsing` pour définir un patron (ou *pattern* en anglais) de ligne qui est appelé `__pattern`. Celui ci est de la forme : `timestamp hostname command[pid]: message` et utilise les fonctions de pyparsing pour une robustesse maximale. On définit ensuite la fonction `parse(self, line)` qui sert à parser une ligne. La ligne est donnée au parseur de patron `__pattern` qui retourne une liste appelée `parsed`. Cette liste est utilisée pour créer un dictionnaire nommé `payload` (qu'on peut traduire par *message utile* en français). Ce dictionnaire à 5 champs (ou `keys`):

* `timestamp` : la date
* `hostname` : le nom de la machine
* `command` : le nom de l'application qui écrit dans le fichier de logs
* `pid` : le PID de l'application (son numéro de processus)
* `message` : le message que l'application à envoyé

La fonction `parseSSH(self, line)` permet de traiter uniquement les lignes générées par le démon `sshd` et de les analyser plus finement. Cette fonction parse la ligne `line` donnée en paramètre et retourne le dictionnaire `payload` qui contient suivant (ceux qui sont facultatifs sont indiqués) :

* `timestamp` : la date
* `pid` : le PID de l'application
* `client` : le nom et la version du client ssh (facultatif)
* `user` : le nom de l'utilisateur utilisé (facultatif)
* `reversedMapping` : vérification du *reverse mapping* (pour éviter les tentatives d'IP spoofing)

Nous allons vérifier si notre parseur fonctionne avec un exemple simple, une ligne tirée du fichier `messages`.

In [3]:
sample_line = "Nov 28 03:50:01 bach cron[16620]: (root) CMD (test -x /usr/sbin/run-crons && /usr/sbin/run-crons)"
parser = SyslogParser()
fields = parser.parse(sample_line)

##### Question

Affichez la ligne à traiter et la ligne traitée pour vérifier si le code à bien fonctionné

In [9]:
print (sample_line)
print (fields)

print ("Le", fields['timestamp'], 
       "le programme", fields['command'], 
       "a ecrit : ", fields['message'])

Nov 28 03:50:01 bach cron[16620]: (root) CMD (test -x /usr/sbin/run-crons && /usr/sbin/run-crons)
{'timestamp': 'Nov 28 03:50:01', 'pid': '16620', 'hostname': 'bach', 'command': 'cron', 'message': '(root) CMD (test -x /usr/sbin/run-crons && /usr/sbin/run-crons)'}
Le Nov 28 03:50:01 le programme cron a ecrit :  (root) CMD (test -x /usr/sbin/run-crons && /usr/sbin/run-crons)


## Analyse des logs

Après avoir vérifié que notre parseur a bien fonctionné, et que tous les champs sont bien renseignés. La ligne parsée génère un dictionnaire, appellé `fields` ici, dont les entrées sont les différents champs.

Avant de traiter le vrai fichier `/var/log/messages`, nous allons utiliser un fichier de test, qui contient les premières lignes de `/var/log/messages`. Cela nous permettra de faire une première analyse, de vérifier le bon fonctionnement sans avoir besoin de traiter l'intégralité des données du vrai fichier. Une fois les tests réalisés, nous pourrons traiter l'intégralité du fichier `message`.

##### Question

Écrivez le code qui permet d'ouvrir le fichier `test_messages5000` et qui parse chacune de ses lignes. Les lignes ainsi parsées  sont à aggréger dans une liste que vous appelerez `log_list`.

Vérifiez la taille de `log_list`, qui doit être égale à 5000 si vous avez ouvert `test_messages5000`.

Il y a bien un nombre de lignes relativement faible pour le fichier de test, comparativement aux 200 000 lignes du "vrai" fichier `/var/log/messages`.

Nous allons maintenant utiliser le module `pandas` pour mettre en forme les données. Ce module permet de créer des *DataFrames* : ce sont des conteneurs qui permettent de mettre l'information en forme.

In [ ]:
import pandas as pd
from pandas import Series, DataFrame, Panel
df = DataFrame(log_list)

##### Question

Affichez les 3 premières éléments de `df`. Les *DataFrame* peuvent se manipuler comme des listes.

Nous allons maintenant utiliser le champ `timestamp` comme étiquettes temporelles pour nos logs :

In [ ]:
# df.index = pd.to_datetime(df.pop(df['timestamp'])) ne fonctionne pas avec pandas 1.7
from datetime import datetime

ts = []
for s in df.timestamp:
    ts.append(datetime.strptime("2014 "+s, "%Y %b %d %H:%M:%S"))
df.index = pd.to_datetime(ts)
del df['timestamp']

Il est possible d'afficher les informations sur cette *DataFrame* avec:

In [ ]:
df.info()

Il est également possible d'afficher les informations sur le contenus et les différents champs de la *DataFrame* :

In [ ]:
df.describe()

Regardons maintenant quels sont les applications qui ont envoyées des messages, nous allons utiliser la fonction `value_counts` pour afficher l'ensemble des valeurs prises par le champs `command`.

In [ ]:
df.command.value_counts()

##### Question

Quelle est la commande la plus fréquente ? Que pouvez vous dire sur les tâches de `cron` ?

## Figures et visualisations

On peut générer un histogramme à partir de ces données de la façon suivante. On regroupe les informations de la *DataFrame* par commande. On calcule ensuite le nombre de chaque commande avec la fonction `size()` puis on affiche un diagramme en bâtons (*bars* en anglais). La bibliothèque Pandas permet de générer facilement toute sort de graphique en appelant la fonction `plot()` avec comme argument `kind`. Les types possibles sont: 

* `bar` ou `barh` pour les diagrammes (verticaux ou horizontaux)
* `hist` pour les histogrammes
* `box` pour les boîtes à moustaches
* `kde` ou `density` pour les figures de densité de probabilités
* `area` pour les aires 
* `scatter` pour les nuages de points
* `hexbin` pour les cartes hexagonales
* `pie` pour les camemberts

In [ ]:
grouped_status = df.groupby('command')
grouped_status.size().plot(kind='bar')

Regardons maintenant le nombres de messages générés toutes les dix minutes. On va donc ré-échantilloner les commandes avec `resample()` en faisant des paquets de 10 minutes (indiqué ici par `10t`) dans lesquels on va sommer chaque commande (avec `how='count'`). 

In [ ]:
df_s = df.command.resample('10t', how='count')
df_s.plot()

Pour voir la part respectives de chaque applications dans la génération de log, on peut séparer les données quantitative pour chaque application puis les regrouper dans un *DataFrame* commun. On peut en profiter pour regrouper les activités de `cron` de `run-crons` sous la même étiquette. Pour avoir des résultats interprétables, on aggrège les messages par groupe de 2h (`120t`).

In [ ]:
t_span = '120t'
df_cron = df['command'][df['command'].isin(['cron', 'run-crons']) ].resample(t_span, how='count')
df_sshd = df['command'][df['command'] == 'sshd'].resample(t_span, how='count')
df_syslog = df['command'][df['command'] == 'syslog-ng'].resample(t_span, how='count')

status_df = DataFrame({'cron': df_cron, 
                       'sshd': df_sshd, 
                       'syslog': df_syslog})
status_df.plot(kind="bar", stacked=True, color=("r", "g","b"), xticks=())

## Analyse des traces SSH

Voici un exemple de traces SSH : 

In [ ]:
samples=[
"Nov 28 05:01:44 bach sshd[17038]: SSH: Server;Ltype: Version;Remote: 122.225.97.104-29185;Protocol: 2.0;Client: libssh2_1.4.2", 
"Nov 28 05:01:45 bach sshd[17038]: SSH: Server;Ltype: Kex;Remote: 122.225.97.104-29185;Enc: aes128-ctr;MAC: hmac-sha1;Comp: none [preauth]",
"Nov 28 05:01:59 bach sshd[17038]: SSH: Server;Ltype: Authname;Remote: 122.225.97.104-29185;Name: root [preauth]",
"Nov 28 05:01:59 bach sshd[17038]: reverse mapping checking getaddrinfo for mx7.fund123.cn [122.225.97.104] failed - POSSIBLE BREAK-IN ATTEMPT!",
"Nov 28 05:01:59 bach sshd[17038]: User root from 122.225.97.104 not allowed because not listed in AllowUsers",
"Nov 28 05:01:59 bach sshd[17038]: input_userauth_request: invalid user root [preauth]",
"Nov 28 05:01:59 bach sshd[17038]: Connection closed by 122.225.97.104 [preauth]"]

On peut voir que les informations sur la connexion SSH sont réparties sur plusieurs messages :

* Le premier indique l'adresse IP, la version du protocole, le client. 
* Le second précise l'encryptage et les moyens cryptographiques. 
* Le troisième donne l'utilisateur utilisé pour la connexion.
* Le quatrième indique que le nom de la machine ne correspond pas à l'adresse IP.
* Le cinquième explique que `root` n'est pas admis à se connecter en SSH.
* Le sixième indique le message envoyé au client.
* Le septième précise que la connexion a été fermée par le client.

Ces sept message sont issus du même processus, on peut voir que le PID est le même pour tous. Or le parseur SSH permet seulement d'analyser les logs lignes à lignes. Le résultat est le suivant :

In [ ]:
parser = SyslogParser()
for s in samples:
    print ("Ligne parsée :\n", parser.parseSSH(s))

Le parseur retourne des informations différentes pour les 4 premières lignes et ne retourne rien pour les 3 dernières.

##### Question

Écrivez le code nécessaire pour traiter le fichier `test_message100` et :

* Créer une liste appelée `sshlog_list` qui va stocker toutes les lignes parsées
* Supprimer les élements vide de la liste (leur valeur est `None`)
* Traiter les éléments de la liste pour fusionner ceux qui ont le même PID. Vous utiliserez `a.update(b)` qui permet de mettre à jour les champs de `a` avec ceux de `b` et `del b` qui permet du supprimer `b`. 

##### Question

Après avoir validé votre code sur la question précédente, adaptez le pour traiter le fichier `test_messages5000`.

Convertissons maintenant la liste obtenue sur ce fichier en *DataFrame* :

In [ ]:
df = DataFrame(sshlog_list)
ts = []
for s in df.timestamp:
    ts.append(datetime.strptime("2014 "+s, "%Y %b %d %H:%M:%S"))
df.index = pd.to_datetime(ts)
del df['timestamp']

Vous pouvez utiliser `df.info()` et `df.describe()` pour vérifier les informations sur la *DataFrame*.

In [ ]:
df.describe()

In [ ]:
df.info()

##### Question

Utilisez la fonction `plot()` pour mettre sous forme de figures les informations de votre fichier de message. Vous proposerez une petite analyse des informations ainsi obtenues.

##### Question (*)

Cette question est facultative. Vous utiliserez le module Basemap pour afficher une carte sur monde sur laquelle vous afficherez les positions géographiques associées à ces adresses IP. Pour cela, vous utiliserez le code suivant qui permet de récuperer les coordonnées de latitude et de longitude pour chaque IP. Le site freegeoip.net qui propose ce service limite le nombre de demandes. Vous devrez donc d'abord enlever les doublons, c'est à dire éviter de faire plusieurs requêtes pour la même IP. 

In [ ]:
import re
from csv import reader
from urllib.request import urlopen # drop request for python 2 version
from codecs import iterdecode
FREE_GEOIP_CSV_URL = "http://freegeoip.net/csv/%s"
def valid_ip(ip):

    pattern = r"\b(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\b"

    return re.match(pattern, ip)

def __get_geodata_csv(ip):
    if not valid_ip(ip):
        raise Exception('Invalid IP format', 'You must enter a valid ip format: X.X.X.X')

    URL = FREE_GEOIP_CSV_URL % ip
    httpstream = urlopen(URL)
    response_csv = reader(iterdecode(httpstream, 'utf-8'))
#    response_csv = reader(urlopen(URL))
    csv_data = next(response_csv)    

    return {
        "status": u"True" == csv_data[0],
        "ip":csv_data[1],
        "countrycode":csv_data[2],
        "countryname":csv_data[3],
        "regioncode":csv_data[4],
        "regionname":csv_data[5],
        "city":csv_data[6],
        "zipcode":csv_data[7],
        "latitude":csv_data[8],
        "longitude":csv_data[9]
    }

def get_geodata(ip):
    return __get_geodata_csv(ip)

Écrivez le code qui permette de récupérer `lats`, la liste des latitudes,
et `lons`, la liste des longitudes, associées aux IP uniques:


In [ ]:
from mpl_toolkits.basemap import Basemap
from matplotlib.pyplot import figure
from numpy import log

m = Basemap(projection='mill') # creation de la carte en utilisant la projection de Millner
figure(figsize=(12,16)) # on change la taille de la figure
x, y = m(lons, lats) # lons et lats sont les listes des latitudes et des longitudes des IP
m.scatter(x, y, s=10*log(ipnb**10), marker='o', color='r') # on affiche des points au coordonnées des IP
m.drawcoastlines(color='SaddleBrown') # on affiche les côtes
m.drawmapboundary(fill_color='SkyBlue') # on remplit les océans
_ = m.fillcontinents(color='BurlyWood',lake_color='SkyBlue', zorder=0) # on remplit les continents